In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
HTML("""<style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>""")

import pandas as pd

import pyarrow
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal as sig
import datetime

import plotly.express as px
import kaleido

import glob
from pathlib import Path

In [2]:
# !git repo clone "https://github.com/SamuelAmrich/Aspis_sam.git"
import Aspis_BinLib as BinLib

In [3]:
binlib = BinLib.BinLib()

In [4]:
lat_bin, lon_bin = np.loadtxt("Bins_equidistant.txt", delimiter='\t', usecols=(0, 1), unpack=True)

In [5]:
glob.glob('data_germany/*.gzip')
data_dir = Path('data_germany/')
data_all = pd.concat(pd.read_parquet(parquet_file) for parquet_file in data_dir.glob('*.gzip'))

In [6]:
data_all = data_all[data_all["Elevation"]>=20]
data_all = data_all.dropna(subset=['Azimuth', 'Elevation', "S4_Sig1"])
data_all = data_all[data_all["S4_Sig1"]>0]
data_all = data_all[data_all["Phi01_Sig1"]>0]
data_all = data_all.sort_index()
data_all.index = pd.to_datetime(data_all.index)
data_all

,WN,TOW,PRN(GSV),RxState,Azimuth,Elevation,AvgSig1_C/N0,S4_Sig1,Cor_S4_Sig1,Phi01_Sig1,...,n_of_input_S4_Sig2_pcode,n_of_input_Phi60_Sig2_pcode,n_of_input_S4_Sig5,n_of_input_Phi60_Sig5,Avg_space_vehicle_x,Avg_space_vehicle_y,Avg_space_vehicle_z,Station,Latitude,Longitude
2015-01-01 00:01:00,1825,345660.0,2,NaN,115.93300,38.8994,52.8448,0.031751,NaN,0.031308,...,3000.0,3000.0,NaN,NaN,8405470.0,19914100.0,16072200.0,Kiruna_50Hz,67.8,20.4
2015-01-01 00:01:00,1825,345660.0,81,NaN,146.92600,70.2701,56.7941,0.020380,NaN,0.013895,...,3000.0,3000.0,NaN,NaN,21586400.0,4139630.0,13003400.0,Toulouse_50Hz,43.6,1.5
2015-01-01 00:01:00,1825,345660.0,79,NaN,8.16951,68.9930,53.4263,0.026373,NaN,0.018071,...,3000.0,3000.0,NaN,NaN,13019900.0,1328510.0,21930400.0,Toulouse_50Hz,43.6,1.5
2015-01-01 00:01:00,1825,345660.0,78,NaN,105.12800,41.3685,47.5772,0.053923,NaN,0.035626,...,3000.0,3000.0,NaN,NaN,17007100.0,15515500.0,10897000.0,Toulouse_50Hz,43.6,1.5
2015-01-01 00:01:00,1825,345660.0,29,NaN,-160.30800,57.0632,53.0105,0.031888,NaN,0.016744,...,3000.0,3000.0,NaN,NaN,24870500.0,-3202930.0,8724080.0,Toulouse_50Hz,43.6,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-08 00:00:00,1826,345600.0,79,NaN,-42.59380,38.7680,52.8028,0.037633,NaN,0.020618,...,3000.0,3000.0,NaN,NaN,4458570.0,-10366400.0,22877800.0,Neustrelitz_50Hz,53.3,13.1
2015-01-08 00:00:00,1826,345600.0,81,NaN,-150.34200,23.3271,51.3432,0.071056,NaN,0.021737,...,3000.0,3000.0,NaN,NaN,25066300.0,-4589500.0,1545510.0,Neustrelitz_50Hz,53.3,13.1
2015-01-08 00:00:00,1826,345600.0,87,NaN,42.79970,42.9825,54.0578,0.044285,NaN,0.017842,...,3000.0,3000.0,NaN,NaN,913898.0,10797600.0,23086500.0,Neustrelitz_50Hz,53.3,13.1
2015-01-08 00:00:00,1826,345600.0,2,NaN,76.70180,40.1060,51.5596,0.042743,NaN,0.017781,...,3000.0,3000.0,NaN,NaN,5261500.0,18325200.0,19009500.0,Neustrelitz_50Hz,53.3,13.1


In [9]:
data = data_all[data_all.index>"2015-01-07 18:00:00"]
data = data[data.index<"2015-01-08 00:00:00"]
data = binlib.add_latlon(data, 3)
data = binlib.calc_dataframe_bins(data)
data_map = binlib.recalc_dataframe(data, name_of_value = "Phi60_Sig1", func = "max")
data, data_map

C:\Users\exped\Downloads\New folder\E\Aspis\Aspis_BinLib.py:327: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\exped\Downloads\New folder\E\Aspis\Aspis_BinLib.py:332: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(                       WN       TOW  PRN(GSV)  RxState   Azimuth  Elevation  \
 2015-01-07 18:01:00  1826  324060.0         5      NaN -168.8680    41.2392   
 2015-01-07 18:01:00  1826  324060.0        84      NaN   92.0501    20.2965   
 2015-01-07 18:01:00  1826  324060.0        83      NaN   34.2955    35.1837   
 2015-01-07 18:01:00  1826  324060.0        75      NaN -134.0000    72.5053   
 2015-01-07 18:01:00  1826  324060.0        74      NaN   71.6208    47.5870   
 ...                   ...       ...       ...      ...       ...        ...   
 2015-01-07 23:59:00  1826  345540.0        81      NaN -162.1380    36.5584   
 2015-01-07 23:59:00  1826  345540.0        79      NaN  -37.6315    35.3089   
 2015-01-07 23:59:00  1826  345540.0        78      NaN   41.9384    74.8052   
 2015-01-07 23:59:00  1826  345540.0        77      NaN  119.4140    31.1489   
 2015-01-07 23:59:00  1826  345540.0        87      NaN   42.9134    43.4930   
 
                      AvgSig1_C/N0   S

In [11]:
binlib.show_globe_map(data_map, "Phi60_Sig1", "count", "lat", "lon", center=(55, 15), zoom=(30, 30), save=False)